In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from UliEngineering.EngineerIO import format_value
from si_prefix import si_format
import plecs_helper as helper
%matplotlib
%matplotlib inline

# Imports and setup
from pint import UnitRegistry
from scipy.signal import find_peaks
from scipy.optimize import fsolve

# pandas display using scientific notation
# pd.set_option('display.float_format', lambda x: f'{x:.3e}')

# use pint
units = UnitRegistry()
units.default_format = "~P.2f"

Using matplotlib backend: <object object at 0x0000023FA3E71520>


C:\Users\ieykamp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [29]:
def parallel(x, y):
    return 1 / (1 / x + 1 / y)

class FerriteCore():
    def __init__(self, inner_diameter, outer_diameter, length, turns, mu_r = 3500) -> None:
        # all quantities passed in should have pint units attached
        self.inner_diameter = inner_diameter
        self.outer_diameter = outer_diameter
        self.length = length
        self.turns = turns

        self.r = (self.inner_diameter + self.outer_diameter) / 2
        self.A = (self.outer_diameter - self.inner_diameter) * self.length

        # mu_0 = 4 * np.pi * 1e-7 * units.tesla * units.meter / units.amp # units.magnetic_constant
        self.mu = units.mu_0 * mu_r

        self.ferrite_permeance = self.mu * self.A / (2 * np.pi * self.r)
        self.gap_length = 2 * units.thou
        self.gap_permeance = units.mu_0 * self.A / self.gap_length
        self.total_permeance = parallel(self.ferrite_permeance, self.gap_permeance)
        self.inductance = self.turns ** 2 * self.total_permeance

        print(f"ferrite premeance: {self.ferrite_permeance.to_compact(units.henry)}")
        print(f"gap premeance: {self.gap_permeance.to_compact(units.henry)}")
        print(f"total premeance: {self.total_permeance.to_compact(units.henry)}")
        print(f"inductance: {(self.inductance).to_compact(units.henry)}")


    def v_sense(self, I0, freq):
        dIdt = I0 * freq
        B = self.mu * I0 / (2 * np.pi * self.r)
        print(f"Your B value is {B.to_compact(units.tesla)}. The maximum is on the order of 530mT.")
        deltaV = self.turns * self.A / (2 * np.pi * self.r) * self.mu * dIdt
        return deltaV

V0 = 0.5 * units.volt
Rshunt = 100 * units.ohm
I0 = V0 / Rshunt
freq = 1e6 * units.hertz * units.cycle
big_ferrite = FerriteCore(inner_diameter = 0.5 * units.inch, outer_diameter = 0.8 * units.inch, length = 1.156 * units.inch, turns = 60)
print(big_ferrite.v_sense(I0, freq).to_compact(units.volt))


ferrite premeance: 9.49 µH
gap premeance: 5.53 µH
total premeance: 3.50 µH
inductance: 12.58 mH
Your B value is 211.99 µT. The maximum is on the order of 530mT.
17.88 V
